# Minimaximizers

This notebook will be used to test the idea of using the training loss scores to try and generate the inputs that produce them. This will not be accurate for the in between cases but for the various minimua and maxima it could potentially show the inputs that produced them, utilizing the learned relationship between inputs and the loss that it produces. If we then input, say 0, (thought that will likely be a hyper parameter in itself), then it will potentially produce one of the several combinations that can lead to a minimum score although there is also a good chance that the model will learn the limits so we will leave the last layer as a linear layer before the output layer. We'll see. 

In [4]:
import pandas as pd
import numpy as np 
import torch

In [7]:
results = pd.read_csv("./data/model_results.csv", index_col = 0)
results

,Recon Loss,Learning Rate,DetEval,batch size,hidden size,latent size,fixed variance,dropout rate,L2 weight,noise layer std,optimizer,nonlinearity,residual user info,RMSE
0,MSE,0.00005,False,128.0,4096.0,512.0,0.0,0.5,0.00001,0.1,Adam,LeakyRelu,True,1.217142
1,MSE,0.00001,False,32.0,4096.0,1024.0,0.3,0.0,0.00000,0.5,RMSProp,Tanh,True,0.969229
2,MSE,0.00005,True,32.0,2048.0,2048.0,0.5,0.1,0.00000,0.1,Adam,Tanh,True,0.793748
3,Likelihood,0.00005,True,64.0,1024.0,512.0,0.1,0.3,0.00000,0.0,Adagrad,Relu,True,1.728256
4,MSE,0.00010,True,64.0,2048.0,512.0,0.5,0.3,0.00010,0.1,Adam,Relu,True,1.214385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,MSE,0.00001,False,32.0,1024.0,1024.0,0.0,0.0,0.00000,0.0,RMSProp,Relu,False,0.717758
0,MSE,0.00010,False,32.0,1024.0,1024.0,0.0,0.5,0.00000,0.5,Adagrad,Sigmoid,False,1.285313
0,MSE,0.00005,False,128.0,1024.0,2048.0,0.1,0.5,0.00000,0.5,Adagrad,Tanh,False,1.321525
0,MSE,0.00010,True,64.0,4096.0,1024.0,0.3,0.1,0.00000,0.1,Adagrad,LeakyRelu,True,1.048281


# Preprocessing

One hot encode the categorical variables and what not.

In [21]:
X = results.drop('RMSE', axis = 1)
y = results.RMSE
X

,Recon Loss,Learning Rate,DetEval,batch size,hidden size,latent size,fixed variance,dropout rate,L2 weight,noise layer std,optimizer,nonlinearity,residual user info
0,MSE,0.00005,False,128.0,4096.0,512.0,0.0,0.5,0.00001,0.1,Adam,LeakyRelu,True
1,MSE,0.00001,False,32.0,4096.0,1024.0,0.3,0.0,0.00000,0.5,RMSProp,Tanh,True
2,MSE,0.00005,True,32.0,2048.0,2048.0,0.5,0.1,0.00000,0.1,Adam,Tanh,True
3,Likelihood,0.00005,True,64.0,1024.0,512.0,0.1,0.3,0.00000,0.0,Adagrad,Relu,True
4,MSE,0.00010,True,64.0,2048.0,512.0,0.5,0.3,0.00010,0.1,Adam,Relu,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,MSE,0.00001,False,32.0,1024.0,1024.0,0.0,0.0,0.00000,0.0,RMSProp,Relu,False
0,MSE,0.00010,False,32.0,1024.0,1024.0,0.0,0.5,0.00000,0.5,Adagrad,Sigmoid,False
0,MSE,0.00005,False,128.0,1024.0,2048.0,0.1,0.5,0.00000,0.5,Adagrad,Tanh,False
0,MSE,0.00010,True,64.0,4096.0,1024.0,0.3,0.1,0.00000,0.1,Adagrad,LeakyRelu,True


In [23]:
X = pd.get_dummies(X)
X

,Learning Rate,DetEval,batch size,hidden size,latent size,fixed variance,dropout rate,L2 weight,Recon Loss_Likelihood,Recon Loss_MSE,...,optimizer_Adagrad,optimizer_Adam,optimizer_RMSProp,nonlinearity_LeakyRelu,nonlinearity_Relu,nonlinearity_Sigmoid,nonlinearity_Tanh,residual user info_False,residual user info_True,residual user info_True.1
0,0.00005,False,128.0,4096.0,512.0,0.0,0.5,0.00001,0,1,...,0,1,0,1,0,0,0,0,1,0
1,0.00001,False,32.0,4096.0,1024.0,0.3,0.0,0.00000,0,1,...,0,0,1,0,0,0,1,0,1,0
2,0.00005,True,32.0,2048.0,2048.0,0.5,0.1,0.00000,0,1,...,0,1,0,0,0,0,1,0,1,0
3,0.00005,True,64.0,1024.0,512.0,0.1,0.3,0.00000,1,0,...,1,0,0,0,1,0,0,0,1,0
4,0.00010,True,64.0,2048.0,512.0,0.5,0.3,0.00010,0,1,...,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.00001,False,32.0,1024.0,1024.0,0.0,0.0,0.00000,0,1,...,0,0,1,0,1,0,0,1,0,0
0,0.00010,False,32.0,1024.0,1024.0,0.0,0.5,0.00000,0,1,...,1,0,0,0,0,1,0,1,0,0
0,0.00005,False,128.0,1024.0,2048.0,0.1,0.5,0.00000,0,1,...,1,0,0,0,0,0,1,1,0,0
0,0.00010,True,64.0,4096.0,1024.0,0.3,0.1,0.00000,0,1,...,1,0,0,1,0,0,0,0,1,0


# Set up dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import nn 
import torch
from sklearn.metrics import precision_recall_fscore_support
device = "cuda"

class PerformanceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
